# Microsoft Employee Attrition Classification Problem

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
data = pd.read_csv('employee-attrition.csv')
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# Splitting data into data type
data.dtypes

Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeNumber               int64
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears   

There are objects and int64 data types.

In [ ]:
# Integer data
data_int = data.select_dtypes(include = ['int64'])
data_int.head()

In [ ]:
scaler = MinMaxScaler()
data_int = scaler.fit_transform(data_int)

In [ ]:
# object data
data_obj = data.select_dtypes(include = ['object'])
data_obj.head()

In [ ]:
# Data Munging
le = LabelEncoder()
data_obj = data_obj.apply(le.fit_transform)
data_obj = data_obj.iloc[:, 1:8]
data_obj.head()

In [ ]:
target = data.iloc[:,[1]]
le = LabelEncoder()
target = target.apply(le.fit_transform)
target.head()

In [ ]:
count = target.apply(pd.value_counts)
print(count)

In [ ]:
# One Hot Encoding
enc = OneHotEncoder()
data_obj = enc.fit_transform(data_obj).toarray()
data_obj.shape

In [ ]:
data = np.hstack([data_obj,data_int])
data.shape

In [ ]:
# Feature Selection
clf = RandomForestClassifier()
clf.fit(data,target)
clf.feature_importances_
model = SelectFromModel(clf, prefit = True)
data = model.transform(data)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2)

In [ ]:
# Stacking Classifier
SGD_regressor = SGDClassifier()
rf = RandomForestClassifier()
gb = LogisticRegression(C = 10)
knn = SVC(C = 0.001)
gbr = GradientBoostingClassifier(max_depth = 3)

In [ ]:
# Meta Estimator
stack = StackingRegressor(regressors=(SGD_regressor,gb, knn,rf),
                            meta_regressor=gbr)

In [ ]:
stack.fit(X_train, y_train)
prediction = stack.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, prediction)
cm = confusion_matrix(y_test, prediction)
print(accuracy)
print(cm)

In [ ]:
cvs = cross_val_score(stack,X_train, y_train, scoring = 'accuracy', cv = 10)
cvs = np.mean(cvs)
print(cvs)

In [ ]:
param_grid = { 'C': [0.001,0.01,0.1,1,10]}
cv = GridSearchCV(knn,param_grid,scoring = 'accuracy',n_jobs = -1, cv = 5 )
cv.fit(X_train, y_train)
best_param = cv.best_params_
print(best_param)